In [284]:
import numpy as np
import math

In [285]:
a_ArAr = 5.311

In [286]:
def gen_simplecubic(nx, ny, nz, lattice_cnst, shiftx = 0, shifty = 0, shiftz = 0):
    return np.array([[i*lattice_cnst+shiftx, j*lattice_cnst+shifty, k*lattice_cnst+shiftz] for i in range(nx) for j in range(ny) for k in range(nz)])

def gen_fcc(nx, ny, nz, lattice_cnst, shiftx = 0, shifty = 0, shiftz = 0):
    return np.array([[[i*lattice_cnst+shiftx, j*lattice_cnst+shifty, k*lattice_cnst+shiftz], [(i+0.5)*lattice_cnst+shiftx, (j+0.5)*lattice_cnst+shifty, k*lattice_cnst+shiftz] ,[(i+0.5)*lattice_cnst+shiftx, j*lattice_cnst+shifty, (k+0.5)*lattice_cnst+shiftz], [i*lattice_cnst+shiftx, (j+0.5)*lattice_cnst+shifty, (k+0.5)*lattice_cnst+shiftz]]for i in range(nx) for j in range(ny) for k in range(nz)]).reshape(-1, 3)

def perturb_cell(cell, strength=0.05):
    return cell + np.random.rand(cell.shape[0], cell.shape[1]) * strength

In [287]:
dim = 2
# cell = gen_simplecubic(dim,dim,dim,a_ArAr)
cell = gen_fcc(dim,dim,dim,a_ArAr)
cell_vector = np.array([[dim*a_ArAr,0,0], [0,dim*a_ArAr,0], [0,0,dim*a_ArAr]])
cell.shape

(32, 3)

In [288]:
def reciprocal_vec(a):
    b = np.empty((3,3))
    vol = np.dot(a[0], np.cross(a[1], a[2]))
    b[0] = np.cross(a[1], a[2]) / vol
    b[1] = np.cross(a[2], a[0]) / vol
    b[2] = np.cross(a[0], a[1]) / vol
    return b

def pbc(d):
    tmp = np.modf(d)[0]
    tmp[tmp > 0.5] -= 1
    tmp[tmp < -0.5] += 1
    return tmp


In [289]:
r_cell_vector = reciprocal_vec(cell_vector)
d = np.array([dim*a_ArAr,11322,-313])
r_trued = pbc(d.dot(r_cell_vector))
trued = r_trued.dot(cell_vector)
d, trued

(array([ 1.0622e+01,  1.1322e+04, -3.1300e+02]),
 array([ 0.   , -1.052, -4.962]))

In [290]:
import pandas as pd
def neighbor_list(xyz, lattice, cutoff=np.Inf):
    nlist = {'atom1':[], 'atom2':[], 'dvec': [], 'r2': []}
    r_lattice = reciprocal_vec(lattice)
    for i in range(len(xyz)):
        for j in range(i+1, len(xyz)):
            d = xyz[j] - xyz[i]
            r_trued = pbc(d.dot(r_cell_vector))
            trued = r_trued.dot(cell_vector)
            r2 = trued.dot(trued)
            if r2 < cutoff**2:
                nlist['atom1'].append(i)
                nlist['atom2'].append(j)
                nlist['dvec'].append(d)
                nlist['r2'].append(r2)
    return nlist

In [291]:
pi = 3.141592654
e_0 = 55.26349406 / 1e4 # e^2 eV^-1 Angstrom^-1, vacuum permittivity
eV = 1.602176634e-19

a_ArAr = 5.311 # Angstrom, lattice constant
epsilon_ArAr = 0.0104 # eV, potential params 
sigma_ArAr = 0.34 * 10 # Angstrom, potential params 

En_Ar = -0.08 # eV, reference cohesive energy

def LJ(r2, epsilon, sigma):
    sq_sigma = sigma * sigma
    s_r_6 = (sq_sigma / r2) * (sq_sigma / r2) * (sq_sigma / r2)
    return 4 * epsilon * (s_r_6 * s_r_6 - s_r_6)

def LJ_slope_prefactor(r2, epsilon, sigma):
    sigma2 = sigma * sigma
    s_r_6 = (sigma2 / r2) * (sigma2 / r2) * (sigma2 / r2)
    return 4 * epsilon * (- 12 * s_r_6 * s_r_6 / r2 + 6 * s_r_6 / r2)

In [292]:
def LJ_Fprime(xyz, lattice):
    nlist_df = pd.DataFrame(neighbor_list(xyz, lattice))
    Fprime = np.zeros((len(xyz), 3))
    pref_df = LJ_slope_prefactor(nlist_df['r2'], epsilon_ArAr, sigma_ArAr)
    for (pref, d, a1, a2) in zip(pref_df, nlist_df['dvec'], nlist_df['atom1'], nlist_df['atom2']):
        Fprime[a1] -= pref * np.array(d)
        Fprime[a2] += pref * np.array(d)
    return Fprime

def nextstep_xyz(xyz, Fprime, rate=1e-6):
    step_xyz = xyz + rate * -Fprime
    return step_xyz

def secant_method(Fprime, step_Fprime, h, rate=1e-6):
    diff = (step_Fprime - Fprime).dot((step_Fprime - Fprime).T).diagonal()
    alpha = -rate * Fprime.dot(h.T).diagonal() / (step_Fprime - Fprime).dot(h.T).diagonal()
    alpha[diff < 1e-16] = 0
    return alpha

def LJ_SD(xyz, lattice):
    Fprime = LJ_Fprime(xyz, lattice)
    step_Fprime = LJ_Fprime(nextstep_xyz(cell, Fprime), cell_vector)
    h = -Fprime
    alpha = secant_method(Fprime, step_Fprime, h)
    next_xyz = np.array([coords + a * -f for (coords, a, f) in zip(xyz, alpha, Fprime)])
    return next_xyz

In [293]:
cell = gen_fcc(dim,dim,dim,a_ArAr)
cell = perturb_cell(cell)
cell_vector = np.array([[dim*a_ArAr,0,0], [0,dim*a_ArAr,0], [0,0,dim*a_ArAr]])
for i in range(10):
    # print(cell[:5])
    nlist_df = pd.DataFrame(neighbor_list(cell, cell_vector))
    print(sum(LJ(nlist_df['r2'], epsilon_ArAr, sigma_ArAr)))
    cell = LJ_SD(cell, cell_vector)

-2.28441129230312
-2.194394305939452
-2.2611857329754255
-1.3419949265246311
-1.9115451410517608
-2.0938400181824437
-2.1349894281930517
-2.012785524562303
-1.7350646236852951
1440.682376347875
